<a href="https://colab.research.google.com/github/Napawan2005/NLP---Text-Preprocessing/blob/Test/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [110]:
import pandas as pd
import nltk


In [111]:
df = pd.read_csv('/content/drive/MyDrive/CS BOOK/NLP/spam.csv', encoding='latin1')

### `encoding='latin1'` คืออะไร?

เมื่อคอมพิวเตอร์เก็บข้อมูลที่เป็นตัวอักษร จะมีการเข้ารหัส (encode) ตัวอักษรเหล่านั้นให้อยู่ในรูปแบบของตัวเลข (ไบต์) และเมื่อต้องการอ่านข้อมูลกลับคืนมา ก็จะต้องมีการถอดรหัส (decode) ตัวเลขเหล่านั้นให้กลับมาเป็นตัวอักษรเดิม

- **`encoding`**: เป็นพารามิเตอร์ในฟังก์ชัน `pd.read_csv()` (และฟังก์ชันอื่นๆ ที่เกี่ยวข้องกับการอ่านไฟล์) ที่ใช้เพื่อบอกโปรแกรมว่าไฟล์ที่คุณกำลังอ่านนั้นถูกเข้ารหัสด้วยชุดอักขระแบบไหน
- **`latin1` (หรือ ISO-8859-1)**: เป็นมาตรฐานการเข้ารหัสชุดอักขระแบบหนึ่งที่นิยมใช้กันในอดีต โดยเฉพาะกับภาษาในกลุ่มยุโรปตะวันตก มันสามารถรองรับอักขระพิเศษบางตัวได้ดีกว่าการเข้ารหัสแบบ `ascii` แต่ไม่ครอบคลุมอักขระทั่วโลกเหมือน `utf-8`

#### ทำไมถึงต้องใช้ `encoding='latin1'`?

ข้อผิดพลาดที่คุณพบก่อนหน้านี้ (`UnicodeDecodeError: 'utf-8' codec can't decode bytes...`) เกิดขึ้นเพราะว่า `pandas` พยายามอ่านไฟล์ CSV ของคุณโดยใช้การเข้ารหัสแบบ **`utf-8`** ซึ่งเป็นค่าเริ่มต้นและเป็นชุดอักขระที่ครอบคลุมมากที่สุดในปัจจุบัน

แต่ถ้าไฟล์ CSV ของคุณถูกสร้างขึ้นหรือบันทึกด้วยการเข้ารหัสแบบอื่น (เช่น `latin1`) เมื่อ `pandas` พยายามถอดรหัสด้วย `utf-8` ก็จะเจอไบต์ที่ไม่สามารถตีความได้ ทำให้เกิดข้อผิดพลาดขึ้น

การระบุ `encoding='latin1'` เป็นการบอก `pandas` ให้ใช้ชุดอักขระ `latin1` ในการถอดรหัสไฟล์ ทำให้สามารถอ่านไฟล์ที่มีการเข้ารหัสแบบนั้นได้อย่างถูกต้องครับ

In [112]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# วิเคราะห์ลักษณะข้อมูล

---

วิเคราะห์ของ spam ว่าข้อมูลส่วนใหญ่มีลักษณะยังไง

In [113]:
df[df['v1'] == 'spam']

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN,NaN
...,...,...,...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,NaN,NaN,NaN
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,NaN,NaN,NaN
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,NaN,NaN,NaN
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,NaN,NaN,NaN


In [114]:
df[df['v1'] == 'spam'].shape

(747, 5)

---
วิเคราะห์ข้อมูลเพื่อได้ข้อเปรียบเทียบที่ชัดเจนว่า โซนไหนเด่นเรื่องอะไร เพื่อทำความสะอาดข้อมูล

In [115]:
df[df['v1'] == 'ham']

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
...,...,...,...,...,...
5565,ham,Huh y lei...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [116]:
df[df['v1'] == 'ham'].shape

(4825, 5)

---

# ข้อสังเกตุที่ค้นพบเกี่ยวกับ Dataset นี้
- มีลัญลักษณ์ที่ใช้ html ปนอยู่ เช่น <#>(row 44) ทำการลบทิ้ง
- ถ้ามีลิงค์เช่น www.dbuk.net(row 12) หรือhttp://wap.xxxmobilemovieclub.com?n=QJKGIGHJJGCBL( row 15) ให้จำจัดทิ้ง/เปลี่ยนเป็น[URL]
- ถ้ามีตัวอักษรพิเศษและตามด้วยตัวเลขที่ไม่รู้จักให้ทำการจำกัดทิ้ง เช่น å£100,000 (row 12) ทำการจำกัดทิ้งหรือแทนที่ [Grand Total]
- ถ้ามีเบอร์โทรให้เปลี่ยน เช่น 08452810075 เปลี่ยนเป็น [call]
- การมีรหัสย่อแปลกๆเช่น 69888 (row 134) , 87121 (row 133) ทำการจำกัดทิ้งหรือแทนที่ [code]
- มีคำย่อ เช่น lol (ทำ normalization ไหม  ?)
- เปลี่ยนตัวอักษรตัวใหญ่เป็นตัวเล็กทั้งหมด
- เปลี่ยนเป็นตัวที่เป็นอังกฤษอย่างเดียวทั้งหมด ไม่ปนภาษาอื่น

- tokenization
- ลดคำ stopwords
- stemming

---


In [117]:
import pandas as pd
import re
import html
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

##  ฟังก์ชันเปลี่ยนเบอร์โทรศัพท์เป็น [call]

In [118]:
def remove_html(text):
    # แปลง HTML entities เช่น &lt; ให้เป็น < ก่อน
    text = html.unescape(text)
    # ลบ tags ที่อยู่ใน <...>
    return re.sub(r'<[^>]+>', '', text)

---

## ฟังก์ชันเปลี่ยน URL เป็น [URL]

In [119]:
def replace_url(text):
    # แทนที่ http, https, www ด้วย [URL]
    text = re.sub(r'https?://[\w\-\.]+(?:[:\d]*)?(?:/[\w\-\./%]*)*(?:\?[\w\-\.=&%]*)?(?:#[\w\-]*)?', '[URL]', text)
    text = re.sub(r'www?://\\S+|www\\.\\S+|wap\\.\\S+', '[URL]', text)
    return text

---
## ฟังก์ชันจัดการตัวเลขทางการเงินและอักขระพิเศษ

In [120]:
def handle_special_currency(text):
    # จัดการ Encoding ผิดปกติ เช่น å£, æ†¯, 憯 ตามด้วยตัวเลข
    # å£100,000 -> [Grand Total]
    text = re.sub(r'(å£|æ†¯|憯|£)[\d,.]+', ' [Grand Total] ', text)
    return text

---
##  ฟังก์ชันเปลี่ยนเบอร์โทรศัพท์



In [121]:
def replace_phone(text):
    # แทนที่ตัวเลขยาวๆ (10-12 หลัก) ด้วย [call]
    # ใช้ \d{10,12} เพื่อจับเบอร์มือถือและเบอร์บ้าน
    return re.sub(r'\b\d{10,12}\b', ' [call] ', text)

---
## ฟังก์ชันจัดการรหัสย่อ (Short Codes)

In [122]:
def replace_code(text):
    # แทนที่ตัวเลข 4-6 หลัก (เช่น SMS code 87121) ด้วย [code]
    return re.sub(r'\b\d{4,6}\b', '[code]', text)

---
## ฟังก์ชัน Normalize คำย่อ (Slang)

In [123]:
def normalize_abbreviations(text):
    abbr_dict = {
        "lol": "laughing out loud",
        "u": "you",
        "ur": "your",
        "r": "are",
        "c": "see",
        "y": "why",
        "wkly": "weekly",
        "txt": "text",
        "msg": "message",
        "cos": "because",
        "gr8": "great"
    }
    words = text.split()
    # แปลงคำย่อถ้ามีใน Dictionary
    return " ".join([abbr_dict.get(w.lower(), w) for w in words])

## ฟังก์ชันเปลี่ยนเป็นตัวพิมพ์เล็ก (Lowercase)

In [124]:
def to_lowercase(text):
    return text.lower()

---
## ฟังก์ชันกรองเฉพาะภาษาอังกฤษ (Keep English Only)

In [125]:
def keep_english_only(text):
    # เก็บตัวอักษร a-z, ช่องว่าง และ [] สำหรับ Token พิเศษที่เราสร้างไว้
    # ลบตัวเลขโดดๆ หรือภาษาอื่นออก
    return re.sub(r'[^a-z\s\[\]]', ' ', text)

---
## ฟังก์ชัน Tokenization

In [126]:
def tokenize(text):
    # ใช้ NLTK หรือ split() ธรรมดา
    return nltk.word_tokenize(text)

---
## ฟังก์ชันลบ Stopwords

In [127]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    # กรองเอาเฉพาะคำที่ไม่ใช่ stopword
    return [w for w in tokens if w not in stop_words]

---
## ฟังก์ชัน Stemming (ลดรูปคำ)

In [128]:
def apply_stemming(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in tokens]

---
## รวมทุกฟังก์ชันเป็น Pipeline

In [129]:
def preprocess_pipeline(text):
    text = remove_html(str(text))
    text = replace_url(text)
    text = handle_special_currency(text)
    text = replace_phone(text)
    text = replace_code(text)
    text = normalize_abbreviations(text)
    text = to_lowercase(text)
    text = keep_english_only(text)

    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = apply_stemming(tokens)

    return tokens

In [130]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [131]:
df['v2_prepro'] = df['v2'].apply(preprocess_pipeline)

In [132]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,v2_prepro
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"[go, jurong, point, crazi, avail, bugi, n, gre..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,"[ok, lar, joke, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,"[free, entri, weekli, comp, win, fa, cup, fina..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,"[dun, say, earli, hor, see, alreadi, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"[nah, think, goe, usf, live, around, though]"
...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN,"[nd, time, tri, contact, u, [, grand, total, ]..."
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN,"[b, go, esplanad, fr, home]"
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN,"[piti, mood, suggest]"
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN,"[guy, bitch, act, like, interest, buy, someth,..."
